# Kili Tutorial: Exporting a training set
In this tutorial, we will walk through the process of using Kili to export a training. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way, but also to dive into the actual process of iteratively developing real applications in Kili.

Additionally:

For an overview of Kili, visit kili-technology.com
You can also check out the Kili documentation https://cloud.kili-technology.com/docs.
Our goal is to export labels that can predict whether an image contains a Porsche or a Tesla.

The tutorial is divided into four parts:

1. Creating a project
2. Settings up the labeling interface
3. Labeling
4. Exporting the training set with the API

## 1. Creating the project

On Kili homepage (https://cloud.kili-technology.com/label/projects), click on the top right button **Create new**.

Then enter a project title, project description and project type **Image classification (single-class)** and click on save.

![create project](img/export_labels-create_project.png)

The project is created with to images by default so you can start playing with Kili.

## 2. Settings up the labeling interface

When created, click on the project and then on settings.
In settings, change to names of the categories to *Porsche* and *Tesla*.

![update settings](img/export_labels-update_settings.png)

## 3. Labeling

Then click on start labeling and label some images.

![label images](img/export_labels-label_images.png)

## 4. Exporting the training set with the API

To export assets you will need the ID of the project. You find it in the url of you browser when you are on this project.

![copy project id](img/export_labels-copy_project_id.png)

This next cell connect the notebook to the Kili API. You need to update the credentials `email`, `password` and `project_id` before.

In [ ]:
import os

# !pip install kili # uncomment if you don't have kili installed already
from kili.client import Kili

api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT') # If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

This next cell fetchs the assets. Assets are the piece of information you want to annotate. Here images.

In [ ]:
project = kili.create_project(input_type='IMAGE', json_interface={
    "jobs": {
        "JOB_0": {
            "mlTask": "OBJECT_DETECTION",
            "tools": [
                "rectangle"
            ],
            "content": {
                "categories": {
                    "OBJECT_A": {
                        "name": "Object A"
                    },
                },
                "input": "radio"
            }
        }
    }
}, project_type='IMAGE_OBJECT_DETECTION_RECTANGLE', title='Example')
project_id = project['id']
print(f'Go to https://cloud.kili-technology.com/label/projects/{project_id}/overview')

After drawing some labels on the interface, you can launch:

In [ ]:
assets = kili.assets(project_id=project_id, fields=[
    'externalId',
    'labels.createdAt',
    'labels.jsonResponse',
    'labels.labelType'
])
print(len(assets))

This next cell lists the assets of your project and the number of labels per asset:

In [ ]:
for asset in assets:
    external_id = asset['externalId']
    number_of_labels = len(asset['labels'])
    print(f'{external_id}: {number_of_labels}')

This next cell prints for each asset the most recent label with its `labelType` and its `jsonResponse`:

- `labelType`: can be either a REVIEW, a PREDICTION (when produced by a model) or DEFAULT (when produced by a human and not a review).
- `jsonResponse`: the annotations I produced on the GUI

In [ ]:
for asset in assets:
    no_label_for_this_asset = len(asset['labels']) == 0
    if no_label_for_this_asset:
        continue
    sorted_labels = sorted(asset['labels'],
                           key=lambda l: l['createdAt'], reverse=True)
    latest_label = sorted_labels[0]
    print(asset['externalId'])
    print(latest_label['labelType'])
    print(latest_label['jsonResponse'])
    print()

## Simplified format

In some very easy projects, Kili API format can seem overwhelming. That is why we also designed formatters. Accepted formats are:

- `None`: it is Kili API raw format as extensively described [here](https://cloud.kili-technology.com/docs/data-export/data-format)
- `'simple'` or `kili.orm.Annotation.AnnotationFormat.Simple`: it is a reduced format for better readability on simple projects
  - For single-class single-job classification projects, you'll only get in JSON responses the chosen class
  - For single-class single-job named-entity recognition projects, you'll only get in JSON responses the list of the chosen class and the entity
  - For single-class single-job object detection projects, you'll only get in JSON responses the list of the chosen class and the polygon

In [ ]:
for asset in assets:
    for label in asset.labels:
        print(label.json_response(format='simple'))

# Summary

In this tutorial, we accomplished the following:

We introduced the concept of Kili project and asset and demonstrated how to create, set up and use one.
We used the Kili API to extract the labels.
We showed that an an asset can have several labels. And that a label is made of a `labelType` and a `jsonResponse` that contains the annotations.
If you enjoyed this tutorial, check out the other Recipes for other tutorials that you may find interesting, including demonstrations of how to use Kili.

You can also visit the [Kili website](https://kili-technology.com) or [Kili documentation](https://cloud.kili-technology.com/docs) for more info!